# **Pharmaceutical Market Cap Data**

Run on Python 3.12 | No errors | No warnings

Data Source: https://companiesmarketcap.com/

This program is designed to connect to a website that blocks non-browser requests to download a dataset. By adding code that mimics a browser request, the program successfully connects and loads the CSV file directly into a Python DataFrame. Once the data is loaded, the review and cleaning process follows standard procedures.

The primary objective is to compare the combined market capitalization of Eli Lilly and Novo Nordisk to the combined market capitalization of GSK (GlaxoSmithKline), J&J (Johnson & Johnson), Merck, Novartis, Pfizer, and Roche. 

Eli Lilly and Novo Nordisk together have a greater market capitalization than the combined total of the other six companies, and this gap continues to widen. This strong performance is driven primarily by four peptides: dulaglutide (Trulicity), liraglutide (Victoza and Saxenda), semaglutide (Ozempic, Rybelsus, and Wegovy), and tirzepatide (Mounjaro and Zepbound).

In [1]:
# Import packages

# For data manipulation
import pandas as pd

# For making HTTP requests and handling responses
import requests as req

# For creating in-memory text streams
from io import StringIO as sio

In [2]:
# Connect to the website and download the CSV file

# URL of the CSV file
url = 'https://companiesmarketcap.com/?download=csv'

# Set headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Send a GET request
response = req.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Read the content into a pandas DataFrame
    csv_data = sio(response.text)
    df0 = pd.read_csv(csv_data, usecols=['Name', 'marketcap', 'price (USD)'])
    print(df0.head())  # Display the first few rows
else: # If the request was unsuccessful
    print(f"Failed to retrieve the CSV file. Status code: {response.status_code}")

                Name      marketcap  price (USD)
0          Microsoft  3360593281024       452.16
1              Apple  3269996838912       213.25
2             NVIDIA  3109225234432       126.40
3  Alphabet (Google)  2280162328576       185.37
4             Amazon  2014821744640       193.61


In [3]:
# Display basic information about the data 
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8552 entries, 0 to 8551
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         8552 non-null   object 
 1   marketcap    8552 non-null   int64  
 2   price (USD)  8552 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 200.6+ KB


In [4]:
# Specify the rows we want to keep. Delete the rest.

# Create a list of companies to keep
companies_to_keep = ['Eli Lilly', 'GlaxoSmithKline', 'Johnson & Johnson', 'Merck', 'Novartis', 'Novo Nordisk', 'Pfizer', 'Roche']

# Create a mask to filter the rows
mask = df0['Name'].isin(companies_to_keep)

# Drop the rows that are not in the list
df0.drop(df0[~mask].index, inplace=True)

In [5]:
# Display all rows of the dataframe to make sure we have the 8 companies we want and that no values are missing
df0.head(8)

,Name,marketcap,price (USD)
9,Eli Lilly,811498995712,901.260
11,Novo Nordisk,644154523648,143.670
26,Johnson & Johnson,353348780032,146.820
28,Merck,333064503296,131.500
49,Roche,224627216405,278.171
52,Novartis,218466697216,107.070
88,Pfizer,155377909760,27.420
215,GlaxoSmithKline,79931875328,38.870


In [6]:
# Display basic information about the data 
df0.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8 entries, 9 to 215
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         8 non-null      object 
 1   marketcap    8 non-null      int64  
 2   price (USD)  8 non-null      float64
dtypes: float64(1), int64(1), object(1)
memory usage: 256.0+ bytes


In [7]:
# Display the first 8 rows of the dataframe
df0.head(8)

,Name,marketcap,price (USD)
9,Eli Lilly,811498995712,901.260
11,Novo Nordisk,644154523648,143.670
26,Johnson & Johnson,353348780032,146.820
28,Merck,333064503296,131.500
49,Roche,224627216405,278.171
52,Novartis,218466697216,107.070
88,Pfizer,155377909760,27.420
215,GlaxoSmithKline,79931875328,38.870


In [8]:
# Create a new column for Market Cap in Billions USD
df0['Market Cap (Billions USD)'] = df0['marketcap'] / 1000000000

In [9]:
# Round the values in the 'Market Cap (Billions USD)' column to 2 decimal places
df0['Market Cap (Billions USD)'] = df0['Market Cap (Billions USD)'].round(2)

In [10]:
# Drop the old marketcap column as it's no longer needed
df0.drop(['marketcap'], axis=1, inplace=True)

In [11]:
# Rename price (USD) to Share Price and Name to Company
df0.rename(columns={'price (USD)': 'Share Price', 'Name': 'Company'}, inplace=True)

In [12]:
# Add a column for year as a string (e.g. '2024')
df0['Year'] = '2024'

In [13]:
# Reindex the dataframe to restore a sequential index
df0.reset_index(drop=True, inplace=True)

In [14]:
# Rename GlaxoSmithKline to GSK
df0['Company'] = df0['Company'].replace('GlaxoSmithKline', 'GSK')

In [15]:
# Rename Johnson & Johnson to J&J
df0['Company'] = df0['Company'].replace('Johnson & Johnson', 'J&J')

In [16]:
# Sort the dataframe by Company
df0.sort_values(by='Company', inplace=True)

In [17]:
# Reorder the columns
df0 = df0[['Company', 'Year', 'Market Cap (Billions USD)', 'Share Price']]

In [18]:
# Display the first 8 rows of the dataframe
df0.head(8)

,Company,Year,Market Cap (Billions USD),Share Price
0,Eli Lilly,2024,811.50,901.260
7,GSK,2024,79.93,38.870
2,J&J,2024,353.35,146.820
3,Merck,2024,333.06,131.500
5,Novartis,2024,218.47,107.070
1,Novo Nordisk,2024,644.15,143.670
6,Pfizer,2024,155.38,27.420
4,Roche,2024,224.63,278.171


In [19]:
# Write the cleaned data to a CSV file without the index
df0.to_csv('Market Cap Data Cleaned.csv', index=False)